In [1]:
import pandas as pd
import pickle
import glob
import pytz

In [2]:
df_wti_1m = pickle.load(open('./large_files/cl-1m.pkl', 'rb'))

f_paths = sorted(glob.glob('./data/yahoo_finance/*_cl.csv'))
dfs = []
for f_path in f_paths:
    df = pd.read_csv(f_path, parse_dates=['Datetime'])
    dfs.append(df)

df_yfin = pd.concat(dfs, ignore_index=True)
df_yfin = df_yfin.rename({
    'Datetime': 'datetime',
    'Open': 'open',
    'Close': 'close',
    'High': 'high',
    'Low': 'low',
    'Volume': 'volume'
}, axis=1)
df_yfin = df_yfin.drop('Adj Close', axis=1)

df_yfin['datetime'] = pd.to_datetime(df_yfin['datetime'], utc=True)
# df_yfin['datetime'] = df_yfin['datetime'].dt.tz_localize('Etc/GMT+6')
ny_tz = pytz.timezone('America/New_York')

df_yfin['datetime'] = df_yfin['datetime'].dt.tz_convert(ny_tz)
df_yfin['date'] = df_yfin['datetime'].dt.strftime('%Y-%m-%d')
df_yfin['time'] = df_yfin['datetime'].dt.strftime('%H%M')
df_yfin['day'] = df_yfin['datetime'].dt.strftime('%A')

In [3]:
df = pd.concat([df_wti_1m, df_yfin])

In [4]:
test = df[df['datetime'].dt.year == 2023]

In [5]:
df = df.drop_duplicates(keep='first')

In [6]:
pickle.dump(df, open('./large_files/cl-1m-2.pkl', 'wb'))